In [ ]:
import pandas
from pandas import read_csv
from datetime import datetime
from matplotlib import pyplot
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from math import sqrt
from numpy import concatenate
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error

## Data Preparation

In [ ]:
#Load general csv file
#df = pandas.read_csv("reports_data.csv") 
df = pandas.read_csv("garitas_data_newest.csv")

#drop out id column
df.drop('id', axis=1, inplace=True)
# Read garita
df_Ysidro = df[df["port"] == "San Ysidro"] 
df_Ysidro.loc[:,'time'] = pandas.to_datetime(df_Ysidro.loc[:,'time']) #check to_datetime

# Mark b'' values with 0
type_line = ['vehicle_ready', 'vehicle_sentry', 'vehicle_standard', 'pedestrian_ready', 'pedestrian_standard']
for line in type_line:
    #df_calexico[line] = df_calexico[line].replace(["b''"], '0')
    df_Ysidro[line] = df_Ysidro[line].replace([""], '0')

#Fill NA/NaN values
X = df_Ysidro.fillna(-1)[["vehicle_ready", "time"]] #X:pandas.core.frame.Dataframe

#Create new column: time_before
data = X["vehicle_ready"] #id and wait-time

# add previous_time column
time_before = [data.iloc[1]] #locks first time
for i in range(1, len(data) - 1):
    #time_before.append(data.iloc[i + 1]) 
    time_before.append(data.iloc[i - 1]) #new df column with previous time
time_before.append(0) #0 value to last space of time_before array

#Rename vehicle_ready column
X.rename(columns={'vehicle_ready': 'actual_time'}, inplace=True)
#Specify new columns with separated values
X["previous_time"] = time_before   #TODO: CHECK THIS COLUMN
X["weekday"] = X["time"].apply(lambda x: x.weekday()) # 0 Monday , 6 Sunday
X["hour"] = X["time"].apply(lambda x: x.hour)
X["minute"] = X["time"].apply(lambda x: x.minute)
X["second"] = X["time"].apply(lambda x: x.second)
X['day'] =  X["time"].apply(lambda x: x.day)
X['month'] =  X["time"].apply(lambda x: x.month)
X['year'] =  X["time"].apply(lambda x: x.year)

#drop out time (utc format column)
X.drop("time", axis=1, inplace=True) 

X.index.name = 'index'

# summarize first 5 rows
#X.head()

# save to file
#X.to_csv('San_Ysidro_vehicle_ready.csv')

## Plot wait-time

In [ ]:
# load dataset
#df = pandas.read_csv('garita_San Ysidro.csv', header=0, index_col=0)
df = X
#convert each row into a numpy array 
values = df.values 
# specify columns to plot
groups = [0]  
i = 1
# plot each column
fig = pyplot.figure(figsize=(10, 6), dpi=500) #set figure size

for group in groups:
    pyplot.subplot(len(groups), 1, i) #made subplot; might want to plot other columns later
    
    pyplot.plot(values[:500, group]) #plot all waiting times
    
    #pyplot.title(df.columns[group], x=0.2, y=0.8, loc='left')
    pyplot.ylabel('Tiempo espera (min)')
    pyplot.xlabel('Muestra')
    pyplot.title('Garita San Ysidro', x=0.7, y=0.8, loc='left')
    i += 1
pyplot.show()
#np.amax(values[:,0]) data has anormal max of 458 min = 7 hr

#Save image file
#fig.savefig('San_Ysidro_Vehicle_Ready.pdf', format='pdf', dpi=2000, bbox_inches='tight')

## LSTM Data preparation

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]   #shape: returns the dimensions of the array
    #data.shape[1]:  no. of columns of array
    df = pandas.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ..., t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pandas.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# load dataset
dataset = read_csv('San_Ysidro_vehicle_ready.csv', header=0, index_col=0)
values = dataset.values #convert each row into a numpy array 

# integer encode direction  (OUR DATA DOES NOT NEED ENCODING)
#encoder = LabelEncoder()
#values[:,4] = encoder.fit_transform(values[:,4])

# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1)) #FR Desired range of transformed data: min=0,max=1
scaled = scaler.fit_transform(values) #Tranfsorm: Scaling features of 'values' according to feature_range

# frame as supervised learning
# specify the number of lag hours
#NOTE: He has time steps of 1 hour, we have smaller time steps 
#n_hours = 5 #Supose take last 5 hours (Note: it is not actually last five hours, measure intervals tend to vary)
n_hours = 300 
n_features = 9 #Have 9 raw features
reframed = series_to_supervised(scaled, n_hours, 1) #Look at the past with lag observations n_hours
#Input params make it multi-step forecasting, e.g., (scaled,2,2)

# drop columns (Only want to predict actual_waiting_time: var1(t))
#reframed.drop([('var%d(t)' % (j+1)) for j in range(1, n_features)], axis=1, inplace=True) #Last column is var1(t)
#var1 : Is the actual_waiting_time ---> variable-to-predict

## Define and fit model

In [ ]:
#Fit an LSTM on the multivariate input data.

In [ ]:
#Split dataset into train and test sets
values = reframed.values
#n_train_hours = 30 * 24 # 1 months: time lapse of training set (complete 31.7.17 to 12.10.17)
train = values[0:n_hours, :] #Take first n_hours to train (THIS CAN CHANGE IF COLLECT MORE DATA) (array of arrays)
test = values[n_hours:, :] #Rest of time 

# split into input and outputs
n_obs = n_hours * n_features  #All previous obs to predict actual t
train_X, train_y = train[:, 0:n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
#train_X: Take n_hours*n_features columns as INPUT for the obs of all features across the previous n_hours (n_obs)
#train_y: take just the wait_time variable as output (var1(t))

# LSTM input params: reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

In [ ]:
#5 Step Life-Cycle for Long Short-Term Memory Models in Keras

# 1 DESIGN NETWORK
model = Sequential() #container of neural network layers Sequential class
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2]))) #The input to every LSTM layer must be three-dimensional!
#50 neurons hidden layers, timesteps, features

model.add(Dense(1)) # 1 neuron in the fully connected output layer with a linear (default) activation function

# 2 COMPILE NETWORK (CHECK: LOSS AND OPTIMIZER OF A MODEL)
#model.compile(loss='mae', optimizer='adam')    
model.compile(loss='mean_squared_error', optimizer='adam')

#ADAM optimization algorithm: for first-order gradient-based optimization of stochastic objective functions
#Mean Absolute Error: loss function
#The lower the Loss, the better a model 

#Question: Why do we get loss for train and validation (test) ???

# 3 FIT NETWORK (adapt the weights of training dataset)
history = model.fit(train_X, train_y, epochs=50, batch_size= len(train_X), validation_data=(test_X, test_y), verbose=2, shuffle=False) 

#history object provides a summary of the performance of the model during training
#Epoch: A full pass over all of your training data. 

# plot history
fig = pyplot.figure(figsize=(5, 3), dpi=800) #set figure size

pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.ylabel('Pérdida')
pyplot.xlabel('Epoch')
pyplot.legend()
pyplot.show()

#fig.savefig('San_Ysidro_300_hour_50_epochs.png', format='png', dpi=900, bbox_inches='tight')

## Evaluate model

In [ ]:
# 4 EVALUATE NETWORK
#Returns the loss value & metrics values for the model in test mode.
loss = model.evaluate(train_X, train_y, verbose=0)
print(loss)

# 5. make predictions
#predictions = model.predict(X, verbose=0)
#print(predictions[:, 0])

# 5 MAKE PREDICTIONS
yhat = model.predict(test_X) #yhat predicted value (done on the test set) . yhat compares with y_test

In [ ]:
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features)) #Check this reshaping

# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -8:]), axis=1) #Ours is 8 (got 9 features)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -8:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

#Calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
points = 490
fig = pyplot.figure(figsize=(10, 6), dpi=500) #set figure size
pyplot.plot(range(points), inv_y[0:points], label='Test')
pyplot.plot(range(points), inv_yhat[0:points], label='Prediction')
#pyplot.plot(range(0,points), test_y[0:points], '-go')
#pyplot.plot(range(0,points), yhat[0:points], '-bo')
pyplot.ylabel('Wait time (min)')
pyplot.xlabel('Sample')
pyplot.legend()
pyplot.show()
#fig.savefig('San_Ysidro_prediction_300_hour_50_epochs.png', format='png', dpi=650, bbox_inches='tight')